Step1: Clean the raw data
Step2: Map ORM

In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import func
from sqlalchemy import Column, Integer, String, Float, Date
from sqlalchemy.orm import Session

import pymysql
pymysql.install_as_MySQLdb()

In [2]:
path1 = "./Resources/hawaii_measurements.csv"
path2 = "./Resources/hawaii_stations.csv"

Step 1 - cleaning data

In [3]:
# The stations are close, and their data can compliment each other. Therefore it is appropriate to remove the missing data. 

def load_data(path):
    df = pd.read_csv(path)
    return df

In [4]:
def clean_export(df, file_name):
    df = df.dropna(axis=0, how='any')
    df.to_csv(file_name, index=False)
    return True

In [5]:
measure = load_data(path1)
measure.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [6]:
clean_export(measure, "clean_measurement.csv")

True

In [7]:
station = load_data(path2)
station.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [8]:
clean_export(station, "clean_station.csv")

True

Step 2 - mapping ORM

In [9]:
clean_measure = pd.read_csv("clean_measurement.csv")
clean_measure.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [10]:
clean_station = pd.read_csv("clean_station.csv")
clean_station.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [11]:
Base = declarative_base()

In [12]:
class Measurement(Base): 
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Integer)


In [13]:
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float) 
    elevation = Column(Float) 

In [14]:
Base.metadata.tables

immutabledict({'measurement': Table('measurement', MetaData(bind=None), Column('id', Integer(), table=<measurement>, primary_key=True, nullable=False), Column('station', String(length=255), table=<measurement>), Column('date', Date(), table=<measurement>), Column('prcp', Float(), table=<measurement>), Column('tobs', Integer(), table=<measurement>), schema=None), 'station': Table('station', MetaData(bind=None), Column('id', Integer(), table=<station>, primary_key=True, nullable=False), Column('station', String(length=255), table=<station>), Column('name', String(length=255), table=<station>), Column('latitude', Float(), table=<station>), Column('longitude', Float(), table=<station>), Column('elevation', Float(), table=<station>), schema=None)})

In [15]:
engine = create_engine("sqlite:///hawaii.sqlite")
conn = engine.connect()

Base.metadata.create_all(engine)


In [16]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement', 'station']

In [17]:
clean_measure.to_sql(name=Measurement.__tablename__, con=conn, index=False, if_exists='append')
clean_station.to_sql(name=Station.__tablename__, con=conn, index=False, if_exists='append')

In [18]:
session = Session(bind=engine)
mm = session.query(Measurement).first()
mm.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1dc1169f0b8>,
 'date': datetime.date(2010, 1, 1),
 'id': 1,
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65}

In [19]:
ss = session.query(Station).first()
ss.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1dc1200a5f8>,
 'elevation': 3.0,
 'id': 1,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}